Redes neurais de classificação geralmente têm uma camada final com uma função sigmoid (para 1 saída) ou softmax (para múltiplas classes). Essa função entrega uma probabilidade entre 0 e 1 para cada classe.

Peça a saída raw da última camada da rede, antes de tomar a decisão.
Essa saída é a probabilidade da classe positiva (euploide).
Multiplique por 100 para transformar em porcentagem.

In [2]:
import pandas as pd
import joblib

# === 1. Carrega a planilha com os dados dos embriões ===
df = pd.read_excel("PlanilhaNumerica.xlsx")

# === 2. Carrega o modelo e o scaler treinados ===
modelo = joblib.load("melhor_modelo_mlp2.pkl")
scaler = joblib.load("scaler_mlp2.pkl")

# === 3. Prepara os dados ===
X = df.drop(columns=["Ploidia"])  # Remove a resposta real
X_scaled = scaler.transform(X)

# === 4. Faz a predição da classe (0 ou 1) ===
classes_preditas = modelo.predict(X_scaled)

# === 5. Adiciona ao DataFrame original ===
df["Classe_Prevista"] = classes_preditas

# === 6. Salva em nova planilha ===
df.to_excel("Planilha_Com_Classificacao.xlsx", index=False)

print("✅ Classificações salvas em 'Planilha_Com_Classificacao.xlsx'")

✅ Classificações salvas em 'Planilha_Com_Classificacao.xlsx'


## Pode testar isso aqui com a planilha nova de dados 

## Etapa 1: Lógica da Porcentagem

Vamos usar as variáveis morfocinéticas e morfológicas da planilha. Para cada variável:

- Se o valor do embrião estiver **na faixa favorável**, ele ganha a **pontuação cheia proporcional ao peso** daquela variável.
- Se estiver **fora da faixa favorável**, ganha **zero** (ou um valor intermediário, se quisermos refinar depois com mais precisão ou uma função contínua).

A **soma ponderada dos acertos** (variáveis que estiverem dentro da faixa favorável) dará a **porcentagem estimada de euploidia**, variando de **0 a 100**.

## Etapa 1: Lógica da Porcentagem

Vamos usar as variáveis morfocinéticas e morfológicas da planilha. Para cada variável:

- Se o valor do embrião estiver **na faixa favorável**, ele ganha a **pontuação cheia proporcional ao peso** daquela variável.
- Se estiver **fora da faixa favorável**, ganha **zero** (ou um valor intermediário, se quiser refinar depois com mais precisão ou uma função contínua).

A **soma ponderada dos acertos** (variáveis que estiverem dentro da faixa favorável) dará a **porcentagem estimada de euploidia**, variando de **0 a 100**.

---

## Etapa 2: Pesos Normalizados (baseados na correlação de Spearman ou evidência)

Aqui estão os pesos que usaremos, totalizando 100%:

| Variável     | Peso (%) |
|--------------|----------|
| Idade        | 28.25    |
| t5           | 13.56    |
| tB - tSB     | 15.82    |
| cc3          | 15.82    |
| Estágio      | 10.00    |
| KIDScore     | 10.00    |
| Morfologia   | 10.00    |
| s2           | 5.00     |
| cc2          | 5.00     |
| t5 - t2      | 5.00     |
| **Total**    | **100**  |

In [ ]:
# === FUNÇÃO DE VERIFICAÇÃO DAS FAIXAS CLÍNICAS ===
def in_range(valor, minimo, maximo):
    return 1.0 if minimo <= valor <= maximo else 0.0

def calcular_porcentagem(row):
    score = 0.0
    # pesos
    pesos = {
        'Idade': 28.25,
        't5': 13.56,
        'tB_tSB': 15.82,
        'cc3': 15.82,
        'Estágio': 10.00,
        'KIDScore': 10.00,
        'Morfo': 10.00,
        's2': 5.00,
        'cc2': 5.00,
        't5_t2': 5.00
    }

    # pontuações
    score += pesos['Idade'] if row['Idade'] < 30 else (0 if row['Idade'] > 37 else pesos['Idade'] * 0.5)
    score += pesos['t5'] if in_range(row['t5'], 48.8, 56.6) else 0
    score += pesos['tB_tSB'] if row['tB-tSB'] < 5 else 0
    score += pesos['cc3'] if in_range(row['cc3 (t5-t3)'], 11, 14) else 0
    score += pesos['Estágio'] if row['Estágio'] == 5 else 0
    score += pesos['KIDScore'] if row['Kidscore'] >= 8 else 0
    score += pesos['Morfo'] if row['Morfo'] in [4, 5] else 0
    score += pesos['s2'] if in_range(row['s2 (t4-t3)'], 0.75, 5) else 0
    score += pesos['cc2'] if in_range(row['cc2 (t3-t2)'], 7.5, 11.9) else 0
    score += pesos['t5_t2'] if in_range(row['t5-t2'], 21, 32) else 0

    return round(score, 2)

# === APLICAR PARA TODOS OS EMBRIÕES ===
df["Porcentagem_Euploidia_Estimada"] = df.apply(calcular_porcentagem, axis=1)

# === SALVAR A PLANILHA FINAL ===
df.to_excel("Planilha_Com_Classificacao_e_Porcentagem.xlsx", index=False)
print("✅ Arquivo salvo com a porcentagem estimada de euploidia.")

## parei em:
- fazer ele classificar uma planilha, e retornar uma nova planilha somente com os euploides.
- depois fazer a porcentagem de euploidia somente com essa